In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import seaborn as sbn

sbn.set()

In [24]:
from ceruleo.dataset.catalog.CMAPSS import CMAPSSDataset
from ceruleo.transformation.functional.transformers import Transformer
from ceruleo.transformation.features.selection import ByNameFeatureSelector
from ceruleo.transformation.functional.pipeline.pipeline import make_pipeline
from ceruleo.transformation.features.scalers import MinMaxScaler
from ceruleo.dataset.catalog.CMAPSS import sensor_indices
from ceruleo.iterators.utils import true_values

from ceruleo.iterators.utils import true_values
from ceruleo.iterators.shufflers import AllShuffled
from ceruleo.iterators.iterators import WindowedDatasetIterator
from ceruleo.models.keras.dataset import tf_regression_dataset
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from ceruleo.models.keras.dataset import tf_regression_dataset


In [25]:
from sklearn.model_selection import train_test_split





dataset = CMAPSSDataset(train=True, models='FD001')
FEATURES = [dataset[0].columns[i] for i in sensor_indices]
train_dataset, test_dataset = train_test_split(dataset, train_size=0.8)


transformer = Transformer(
    pipelineX=make_pipeline(
        ByNameFeatureSelector(features=FEATURES), 
        MinMaxScaler(range=(-1, 1))

    ), 
    pipelineY=make_pipeline(
        ByNameFeatureSelector(features=['RUL']),  
    )
)

transformer.fit(train_dataset)

transformed_train_dataset_FD001 = train_dataset.map(transformer)
transformed_test_dataset_FD001 = test_dataset.map(transformer)


In [26]:
train_iterator_FD001 = WindowedDatasetIterator(
    transformed_train_dataset_FD001,
    window_size=32,
    step=1,
    horizon=1,
    shuffler=AllShuffled())

test_iterator_FD001 = WindowedDatasetIterator(
    transformed_test_dataset_FD001,
    window_size=32,
    step=1,
    horizon=1)



In [27]:
from sklearn.model_selection import train_test_split

dataset = CMAPSSDataset(train=True, models='FD002')
train_dataset, test_dataset = train_test_split(dataset, train_size=0.8)


transformer = Transformer(
    pipelineX=make_pipeline(
        ByNameFeatureSelector(features=FEATURES), 
        MinMaxScaler(range=(-1, 1))

    ), 
    pipelineY=make_pipeline(
        ByNameFeatureSelector(features=['RUL']),  
    )
)


transformer.fit(train_dataset)
transformed_train_dataset_FD002 = train_dataset.map(transformer)
transformed_test_dataset_FD002 = test_dataset.map(transformer)

train_iterator_FD002 = WindowedDatasetIterator(
    transformed_train_dataset_FD002,
    window_size=32,
    step=1,
    horizon=1,
    shuffler=AllShuffled())

test_iterator_FD002 = WindowedDatasetIterator(
    transformed_test_dataset_FD002,
    window_size=32,
    step=1,
    horizon=1)




In [19]:
from adapt.feature_based import DANN

In [31]:
import tensorflow as tf
model = DANN()
model.fit(X=tf_regression_dataset(train_iterator_FD001), 
          Xt=tf_regression_dataset(train_iterator_FD002))

Computing src dataset size...
Done!
Computing tgt dataset size...
Done!
1144/1144 [==============================] - 178s 91ms/step - loss: 2314.1606 - disc_loss: 0.2618
